## Homework-2: ConvNet for MNIST Classification

### **Deadline: 2019.11.03 23:59:59**

### In this homework, you need to
- #### implement forward and backward for ConvLayer (`layers/conv_layer.py`)
- #### implement forward and backward for PoolingLayer (`layers/pooling_layer.py`)
- #### implement forward and backward for ReshapeLayer (`layers/reshape_layer.py`)

In [1]:
import numpy as np
#import cupy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow import keras


from network import Network
from solver import train, test
from plot import plot_loss_and_acc

C:\Users\chou\AppData\Roaming\Python\Python35\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\chou\AppData\Roaming\Python\Python35\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\chou\AppData\Roaming\Python\Python35\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\chou\AppData\Roaming\Python\Python35\site-pack

## Load MNIST Dataset
We use tensorflow tools to load dataset for convenience.

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [3]:
def decode_image(image):
    # Normalize from [0, 255.] to [0., 1.0], and then subtract by the mean value
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [1, 28, 28])
    image = image / 255.0
    image = image - tf.reduce_mean(image)
    return image

def decode_label(label):
    # Encode label with one-hot encoding
    return tf.one_hot(label, depth=10)

In [4]:
# Data Preprocessing
x_train = tf.data.Dataset.from_tensor_slices(x_train).map(decode_image)
y_train = tf.data.Dataset.from_tensor_slices(y_train).map(decode_label)
data_train = tf.data.Dataset.zip((x_train, y_train))
x_test = tf.data.Dataset.from_tensor_slices(x_test).map(decode_image)
y_test = tf.data.Dataset.from_tensor_slices(y_test).map(decode_label)
data_test = tf.data.Dataset.zip((x_test, y_test))

## Set Hyperparameters
You can modify hyperparameters by yourself.

In [5]:
batch_size = 100
max_epoch = 2
init_std = 0.01

learning_rate = 0.15
weight_decay = 0.005

disp_freq = 10

## Criterion and Optimizer

In [6]:
from criterion import SoftmaxCrossEntropyLossLayer
from optimizer import SGD

criterion = SoftmaxCrossEntropyLossLayer()
sgd = SGD(learning_rate, weight_decay)

## ConvNet

In [7]:
from layers import FCLayer, ReLULayer, ConvLayer, MaxPoolingLayer, ReshapeLayer

convNet = Network()
# Build ConvNet with ConvLayer and PoolingLayer
convNet.add(ConvLayer(1, 8, 3, 1))
convNet.add(ReLULayer())
convNet.add(MaxPoolingLayer(2, 0))
convNet.add(ConvLayer(8, 16, 3, 1))
convNet.add(ReLULayer())
convNet.add(MaxPoolingLayer(2, 0))
convNet.add(ReshapeLayer((batch_size, 16, 7, 7), (batch_size, 784)))
convNet.add(FCLayer(784, 128))
convNet.add(ReLULayer())
convNet.add(FCLayer(128, 10))

In [ ]:
# Train
convNet, conv_loss, conv_acc = train(convNet, criterion, sgd, data_train, max_epoch, batch_size, disp_freq)

Epoch [0][2]	 Batch [0][550]	 Training Loss 6.8729	 Accuracy 0.1200
Epoch [0][2]	 Batch [10][550]	 Training Loss 8.9743	 Accuracy 0.1736
Epoch [0][2]	 Batch [20][550]	 Training Loss 5.5160	 Accuracy 0.3267
Epoch [0][2]	 Batch [30][550]	 Training Loss 4.0861	 Accuracy 0.4381
Epoch [0][2]	 Batch [40][550]	 Training Loss 3.2892	 Accuracy 0.5154
Epoch [0][2]	 Batch [50][550]	 Training Loss 2.7522	 Accuracy 0.5786
Epoch [0][2]	 Batch [60][550]	 Training Loss 2.3760	 Accuracy 0.6261
Epoch [0][2]	 Batch [70][550]	 Training Loss 2.0947	 Accuracy 0.6632
Epoch [0][2]	 Batch [80][550]	 Training Loss 1.9089	 Accuracy 0.6847
Epoch [0][2]	 Batch [90][550]	 Training Loss 1.7368	 Accuracy 0.7069
Epoch [0][2]	 Batch [100][550]	 Training Loss 1.5926	 Accuracy 0.7280
Epoch [0][2]	 Batch [110][550]	 Training Loss 1.4732	 Accuracy 0.7453
Epoch [0][2]	 Batch [120][550]	 Training Loss 1.3726	 Accuracy 0.7598
Epoch [0][2]	 Batch [130][550]	 Training Loss 1.2866	 Accuracy 0.7724
Epoch [0][2]	 Batch [140][550]	

In [ ]:
# Test
test(convNet, criterion, data_test, batch_size, disp_freq)

## Plot

In [ ]:
plot_loss_and_acc({'ConvNet': [conv_loss, conv_acc]})

### You have finished homework-2, congratulations!